# LangGraph Tutorial: Parallel Execution

## Objective
Understand how the agent executes multiple independent tools simultaneously in a single request.

## What You'll Learn
1. When parallel execution occurs (independent tasks)
2. How the LLM requests multiple tools in one AIMessage
3. How ToolNode executes tools concurrently
4. The message pattern for parallel execution (5 messages)
5. Performance benefits of parallel tool calls

## Prerequisites
- Completed: Notebook 06 (Single Tool Execution)
- Understanding of the 4-message single tool pattern

---

## Section 1: The Parallel Execution Pattern

### When Does Parallel Execution Happen?

Parallel execution occurs when:
- Query requires **multiple tools**
- Tasks are **independent** (neither needs the other's result)
- LLM recognizes this and calls all tools at once

### Reference Point: Parallel vs Sequential

| Pattern | Condition | Example |
|---------|-----------|--------|
| **Parallel** | Tasks are independent | "Convert USD to EUR AND calculate EMI" |
| **Sequential** | Task B needs Task A's result | "Convert my loan to EUR, then calculate EMI" |

### Execution Flow

```
Query: Task A AND Task B (independent)
  ↓
Agent: Analyzes query, sees two independent tasks
  ↓
Agent: Returns AIMessage with tool_calls=[Tool A, Tool B]
  ↓
ToolNode: Executes BOTH tools simultaneously (ThreadPoolExecutor)
  ↓
Agent: Receives both results, synthesizes single response
  ↓
END
```

### Reference Point: Message Pattern Comparison

| Execution Type | Message Count | Pattern |
|----------------|---------------|--------|
| **Single Tool** | 4 messages | Human → AI(1 tool) → Tool → AI |
| **Parallel (2 tools)** | 5 messages | Human → AI(2 tools) → Tool → Tool → AI |
| **Parallel (3 tools)** | 6 messages | Human → AI(3 tools) → Tool → Tool → Tool → AI |

**Formula:** `messages = 3 + number_of_tools`

---

## Section 2: Setup

Build the financial assistant graph with both tools.

In [ ]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv
from typing import Literal

load_dotenv("../../.env")
print("✅ Environment loaded")

In [ ]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates or to_currency not in exchange_rates:
        return f"Error: Unsupported currency"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

In [ ]:
# Initialize LLM and build graph
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

def call_llm(state: MessagesState):
    """Agent node: Calls LLM with current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router: Check if agent wants to use tools."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "agent")

app = workflow.compile()
print("✅ Graph compiled")

---

## Section 3: Parallel Execution Example

**Query:** "Convert 100000 USD to EUR AND ALSO calculate EMI for 500000 INR at 8.5% for 60 months"

### Why This Query Triggers Parallel Execution

| Task | Tool Needed | Depends On Other? |
|------|-------------|-------------------|
| Convert 100000 USD to EUR | currency_converter | ❌ No |
| Calculate EMI for 500000 INR | emi_calculator | ❌ No |

**Result:** Tasks are independent → Parallel execution!

In [ ]:
# Create state with parallel query
state = {
    "messages": [
        HumanMessage(content="Convert 100000 USD to EUR AND ALSO calculate EMI for 500000 INR at 8.5% for 60 months")
    ]
}

print("Query Analysis:")
print("=" * 80)
print(f"Query: {state['messages'][0].content}")
print("\nExpected behavior:")
print("  • LLM should identify TWO independent tasks")
print("  • LLM should call BOTH tools in a single request")
print("  • ToolNode should execute them in parallel")
print("=" * 80)

In [ ]:
# Execute the graph
result = app.invoke(state)

print("Execution Complete!")
print("=" * 80)
print(f"Total messages: {len(result['messages'])} (expected: 5 for parallel)")
print("=" * 80)

---

## Section 4: Verify Parallel Tool Calls

The key indicator of parallel execution: **multiple tool_calls in a single AIMessage**.

In [ ]:
# Check the AIMessage that contains tool calls
tool_call_message = result['messages'][1]

print("PARALLEL EXECUTION VERIFICATION")
print("=" * 80)
print(f"Message type: {type(tool_call_message).__name__}")
print(f"Number of tool_calls: {len(tool_call_message.tool_calls)}")

if len(tool_call_message.tool_calls) > 1:
    print("\n🚀 CONFIRMED: Parallel execution detected!")
    print(f"   {len(tool_call_message.tool_calls)} tools called in a SINGLE AIMessage")
else:
    print("\n⚠️  Single tool call detected (not parallel)")

print("\n" + "-" * 80)
print("Tool Calls Detail:")
print("-" * 80)
for i, tc in enumerate(tool_call_message.tool_calls, 1):
    print(f"\n  Tool Call {i}:")
    print(f"    Name: {tc['name']}")
    print(f"    Args: {tc['args']}")
    print(f"    ID:   {tc['id']}")

### Reference Point: Single AIMessage with Multiple tool_calls

```python
# Parallel execution signature:
AIMessage(
    content="",
    tool_calls=[
        {"name": "currency_converter", "args": {...}, "id": "call_1"},
        {"name": "emi_calculator", "args": {...}, "id": "call_2"}
    ]
)

# vs Sequential execution (separate messages):
AIMessage(tool_calls=[{"name": "tool_1", ...}])  # First call
# ... tool executes ...
AIMessage(tool_calls=[{"name": "tool_2", ...}])  # Second call (needs first result)
```

---

## Section 5: Examine the Complete Message Flow

In [ ]:
print("COMPLETE MESSAGE FLOW")
print("=" * 80)

for i, msg in enumerate(result["messages"], 1):
    print(f"\n{'─' * 80}")
    print(f"MESSAGE {i}: {type(msg).__name__}")
    print(f"{'─' * 80}")
    
    if isinstance(msg, HumanMessage):
        print(f"  👤 USER INPUT")
        print(f"  Content: {msg.content[:60]}...")
        
    elif isinstance(msg, AIMessage):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"  🤖 AGENT: Requesting {len(msg.tool_calls)} tool(s) IN PARALLEL")
            for tc in msg.tool_calls:
                print(f"    • {tc['name']}")
        else:
            print(f"  🤖 AGENT: Final synthesized response")
            print(f"  Content: {msg.content[:100]}...")
            
    elif isinstance(msg, ToolMessage):
        # Find which tool this result belongs to
        tool_name = "unknown"
        for tc in tool_call_message.tool_calls:
            if tc['id'] == msg.tool_call_id:
                tool_name = tc['name']
                break
        print(f"  🔧 TOOL RESULT: {tool_name}")
        print(f"  Tool Call ID: {msg.tool_call_id}")
        # Show first line of result
        first_line = msg.content.split('\n')[0]
        print(f"  Result: {first_line}...")

### Reference Point: Parallel Execution Message Sequence

```
┌─────────────────────────────────────────────────────────────────────────┐
│ MESSAGE 1: HumanMessage                                                │
│   "Convert 100000 USD to EUR AND calculate EMI for 500000 INR..."     │
└─────────────────────────────────────────────────────────────────────────┘
                                    ↓
┌─────────────────────────────────────────────────────────────────────────┐
│ MESSAGE 2: AIMessage (with 2 tool_calls)                               │
│   tool_calls = [                                                       │
│     {name: "currency_converter", args: {...}},  ←── Both tools        │
│     {name: "emi_calculator", args: {...}}       ←── in ONE message    │
│   ]                                                                    │
└─────────────────────────────────────────────────────────────────────────┘
                                    ↓
              ┌─────────── ToolNode ───────────┐
              │   Executes BOTH in parallel    │
              │   using ThreadPoolExecutor     │
              └─────────────────────────────────┘
                        ↓             ↓
┌─────────────────────────────┐ ┌─────────────────────────────┐
│ MESSAGE 3: ToolMessage      │ │ MESSAGE 4: ToolMessage      │
│   currency_converter result │ │   emi_calculator result     │
└─────────────────────────────┘ └─────────────────────────────┘
                        ↓             ↓
                        └──────┬──────┘
                               ↓
┌─────────────────────────────────────────────────────────────────────────┐
│ MESSAGE 5: AIMessage (final response)                                  │
│   Synthesizes BOTH results into a single natural language answer       │
└─────────────────────────────────────────────────────────────────────────┘
```

---

## Section 6: Deep Dive into Each Message

In [ ]:
# Message 1: User Query
print("MESSAGE 1: HumanMessage (User Query)")
print("=" * 80)
print(f"Content: {result['messages'][0].content}")
print("\n→ User asks for TWO independent things in one query.")

In [ ]:
# Message 2: AIMessage with MULTIPLE tool_calls
print("MESSAGE 2: AIMessage (Parallel Tool Requests)")
print("=" * 80)
msg2 = result['messages'][1]
print(f"Number of tool_calls: {len(msg2.tool_calls)}")
print(f"\nTool Calls:")
for i, tc in enumerate(msg2.tool_calls, 1):
    print(f"\n  [{i}] {tc['name']}")
    print(f"      Args: {tc['args']}")

print("\n→ LLM recognized TWO independent tasks.")
print("→ Requested BOTH tools in a SINGLE AIMessage.")
print("→ This triggers parallel execution in ToolNode.")

In [ ]:
# Messages 3 & 4: Tool Results (executed in parallel)
print("MESSAGES 3 & 4: ToolMessages (Parallel Results)")
print("=" * 80)

for idx in [2, 3]:
    msg = result['messages'][idx]
    print(f"\nMessage {idx + 1}: ToolMessage")
    print(f"  Tool Call ID: {msg.tool_call_id}")
    print(f"  Result:")
    for line in msg.content.split('\n'):
        print(f"    {line}")

print("\n→ Both tools executed simultaneously by ToolNode.")
print("→ Each result has its own ToolMessage.")
print("→ tool_call_id links result to original request.")

In [ ]:
# Message 5: Final Synthesized Response
print("MESSAGE 5: AIMessage (Synthesized Response)")
print("=" * 80)
msg5 = result['messages'][4]
print(f"Content:\n{msg5.content}")

print("\n→ LLM received BOTH tool results.")
print("→ Synthesized into a single coherent response.")
print("→ No more tool_calls = Execution ends.")

---

## Section 7: Streaming View (Real-Time Execution)

Use `.stream()` to see each step as it happens.

In [ ]:
# Reset state and stream execution
state_stream = {
    "messages": [
        HumanMessage(content="Convert 100000 USD to EUR AND ALSO calculate EMI for 500000 INR at 8.5% for 60 months")
    ]
}

print("STREAMING EXECUTION")
print("=" * 80)
print("Watch each node execute in real-time...\n")

step_count = 0
for event in app.stream(state_stream):
    for node_name, data in event.items():
        step_count += 1
        print(f"\n[Step {step_count}] Node: '{node_name}'")
        print("-" * 60)
        
        if "messages" in data:
            for msg in data["messages"]:
                if isinstance(msg, AIMessage):
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        print(f"  🚀 PARALLEL CALL: {len(msg.tool_calls)} tools requested")
                        for tc in msg.tool_calls:
                            print(f"     • {tc['name']}")
                    else:
                        print(f"  💬 Final response generated")
                        
                elif isinstance(msg, ToolMessage):
                    print(f"  ✅ Tool executed (ID: {msg.tool_call_id[:20]}...)")

print("\n" + "=" * 80)
print(f"Total steps: {step_count}")
print("=" * 80)

### Reference Point: Streaming Output Analysis

```
Step 1: agent node
  → LLM analyzes query
  → Returns AIMessage with 2 tool_calls

Step 2: tools node  
  → ToolNode receives 2 tool_calls
  → Executes BOTH using ThreadPoolExecutor
  → Returns 2 ToolMessages

Step 3: agent node
  → LLM sees both tool results
  → Generates final synthesized response
  → No more tool_calls → Routes to END
```

**Key Insight:** Only 3 steps (agent → tools → agent) even with 2 tools!

---

## Section 8: Performance Benefits

### Reference Point: Parallel vs Sequential Performance

| Metric | Sequential (2 tools) | Parallel (2 tools) |
|--------|---------------------|--------------------|
| LLM Calls | 4 | 2 |
| Tool Executions | 2 (one at a time) | 2 (simultaneous) |
| Loop Iterations | 2 | 1 |
| Messages | 6+ | 5 |
| Total Time | ~4 seconds | ~2 seconds |

### Why Parallel is Faster

```
Sequential:                      Parallel:
┌─────────┐                     ┌─────────┐
│ LLM #1  │ 500ms               │ LLM #1  │ 500ms
├─────────┤                     ├─────────┤
│ Tool A  │ 100ms               │ Tool A  │ ┐
├─────────┤                     │ Tool B  │ ┘ 100ms (parallel)
│ LLM #2  │ 500ms               ├─────────┤
├─────────┤                     │ LLM #2  │ 500ms
│ Tool B  │ 100ms               └─────────┘
├─────────┤                     Total: ~1100ms
│ LLM #3  │ 500ms
├─────────┤
│ LLM #4  │ 500ms
└─────────┘
Total: ~2200ms
```

---

## Section 9: How ToolNode Handles Parallel Execution

### Reference Point: ToolNode Internal Behavior

```python
# Simplified ToolNode logic:
def __call__(self, state):
    last_message = state["messages"][-1]
    tool_calls = last_message.tool_calls
    
    if len(tool_calls) == 1:
        # Single tool: execute directly
        result = execute_tool(tool_calls[0])
        return {"messages": [result]}
    else:
        # Multiple tools: execute in parallel
        with ThreadPoolExecutor() as executor:
            results = list(executor.map(execute_tool, tool_calls))
        return {"messages": results}  # Multiple ToolMessages
```

### Key Points

1. **Automatic Detection:** ToolNode checks `len(tool_calls)`
2. **ThreadPoolExecutor:** Python's built-in for concurrent execution
3. **Result Ordering:** Results maintain order of original tool_calls
4. **ID Matching:** Each ToolMessage has `tool_call_id` for correlation

---

## Section 10: LLM's Autonomous Decision-Making

### How the LLM Decides to Use Parallel Execution

The LLM determines independence by analyzing:

| Signal | Example | Interpretation |
|--------|---------|---------------|
| "AND" / "ALSO" | "Convert X AND calculate Y" | Likely independent |
| "both" | "Do both tasks" | Likely independent |
| "then" / "after" | "Convert, then calculate with result" | Dependent (sequential) |
| Data flow | "Convert $X, then calculate EMI for that amount" | Dependent (sequential) |

### No Hardcoded Logic

```
You define:           LLM decides:
─────────────        ────────────────────────────────
• Tool schemas       • Are tasks independent?
• Graph structure    • How many tools to call?
• Router logic       • What parameters to extract?
                     • When to stop?
```

> **Key Insight:** The same query structure can produce parallel or sequential execution depending on semantic analysis.

---

## Summary

In this notebook, you learned:

| Concept | Key Takeaway |
|---------|-------------|
| **When Parallel Occurs** | Tasks are independent (neither needs other's result) |
| **How to Identify** | Single AIMessage with multiple `tool_calls` |
| **Message Count** | 3 + number_of_tools (5 for 2 tools) |
| **Execution** | ToolNode uses ThreadPoolExecutor |
| **Performance** | Fewer LLM calls, faster execution |
| **LLM Autonomy** | Decides independence from natural language |

## Parallel Execution Checklist

```
✅ Multiple tool_calls in ONE AIMessage
✅ Tools execute simultaneously
✅ Multiple ToolMessages returned
✅ Single synthesized final response
✅ Only 2 LLM calls (regardless of tool count)
✅ Only 1 loop iteration (agent → tools → agent → END)
```

## Next Steps

In **Notebook 08: Sequential Execution**, we'll explore:
- Dependent tasks where Task B needs Task A's result
- Multiple loop iterations
- How the LLM chains tool calls

---